In [90]:
import pandas as pd
import numpy as np
import cvxpy


In [161]:
# Cargar las stats de los players
pitch_stats = pd.read_csv("./data/league/pitch_stats.csv", index_col=0)
bats_stats = pd.read_csv("./data/league/bats_stats.csv", index_col=0)

# Import salaries
salaries_batters = pd.read_csv('data/league/batter_salary.csv', index_col=0)
salaries_pitchers = pd.read_csv('data/league/pitcher_salary.csv', index_col=0)

# Add index as a column
pitch_stats['ID'] = pitch_stats.index
bats_stats['ID'] = bats_stats.index

# Replaces with correct salaries
pitch_stats.Salary = salaries_pitchers.Salary
bats_stats.Salary = salaries_batters.Salary

In [92]:
bats = pd.read_csv("data/league/my_team_batters.csv", header=1)
pitchers = pd.read_csv("data/league/my_team_pitchers.csv", header=1)

# Rename columns Eligible => Position
bats.rename(columns={"Eligible": "Position"}, inplace=True)
pitchers.rename(columns={"Eligible": "Position"}, inplace=True)

In [163]:
bats = bats[bats.Status != "Min"]
pitchers = pitchers[pitchers.Status != "Min"]

# Primera Opcion
Keepear un jugador tiene un costo basado en su costo de ronda del draft:
- Minors: 0
- 30 - 21: 1
- 20 -16: 2
- 15 - 11: 3
- 10 - 6: 4
- 5 -  1: 5

Total de non-minors: 10

In [93]:
# Define a function to get the cost of a player
def get_cost(salary):
    if salary >= 21:
        return 1
    elif salary >= 16:
        return 2
    elif salary >= 11:
        return 3
    elif salary >= 6:
        return 4
    else:
        return 5
    
# Assign a cost to each player
bats["cost_1"] = bats["Salary"].apply(get_cost)
# Fix rookie cost to 0
bats.loc[bats.Status == "Min", "cost_1"] = 0
pitchers["cost_1"] = pitchers["Salary"].apply(get_cost)
# Fix rookie cost to 0
pitchers.loc[pitchers.Status == "Min", "cost_1"] = 0

In [94]:
# Compile the data into a single dataframe (Keep only name, cost, position and ID)
players = pd.concat([bats[["Player", "cost_1", "Position", "ID"]], pitchers[["Player", "cost_1", "Position", "ID"]]])

players.sort_values("cost_1")

,Player,cost_1,Position,ID
21,Jasson Dominguez,0,OF,*050c3*
22,Julio Rodriguez,0,OF,*04fhb*
18,Roansy Contreras,0,SP,*04b3a*
20,Kahlil Watson,0,"SS,INF",*05rau*
19,Anthony Volpe,0,"SS,INF",*04y9p*
18,Oswald Peraza,0,"SS,INF",*04g7n*
17,CJ Abrams,0,"2B,SS,INF",*04qk8*
16,Adley Rutschman,0,C,*04q2n*
19,Grayson Rodriguez,0,SP,*04pko*
2,Ryan Helsley,1,RP,*03ql5*


# Segunda Opcion
Usar la ronda como base para el costo de keep:
- Minors:   0
- 30 - 21:  1
- 20 - 16:  5
- 15 - 11: 10
- 10 -  6: 25
-  5 -  1: 40

Luego add performance bonus if more than `170` at bats or `60` innings pitched for starters and `30` innings pitched for relievers.

**Batters:**
- **HR**: 1/6
- **RBI**: 1/10
- **R**: 1/10
- **H**: 1/15
- **SBN**: 1/2
- **SO**: - 1/10
- **BB**: - 1/7
- **AVG**: 100/3
- **OBP**: 100/4
- **SLG**: 100/5

**Pitchers:**
- **IP**: 1/18
- **H**: (200 - x) / 10 if SP (80 - x) / 10 if RP
- **BB**: (80 - x) / 10 if SP (50 - x) / 10 if RP
- **K**:  1/18 if SP 1/8 if RP
- **ER**: (100 - x) / 10 if SP (40 - x) / 10 if RP
- **RA**: 1/6.5
- **QS**: 1/20
- **NSV**: 1/20
- **NSVH**: 1/30

Luego el performance bonus se normaliza de tal foma que el mejor jugador tenga un valor de 20 (Costo total 60 si fue primer round pick) y se aggrega la ronda del draft.

In [160]:
# Define a function to get the base cost of a player
def get_cost_base(salary):
    if salary >= 21:
        return 1
    elif salary >= 16:
        return 5
    elif salary >= 11:
        return 10
    elif salary >= 6:
        return 25
    else:
        return 40
    
# Performance bonus:

## Bats
def get_perf_bonus_bats(player_id, bats_stats):
    # Check if player is in list
    if not (player_id in bats_stats.index):
        return 0
    # Get the player's stats
    player = bats_stats.loc[player_id]
    # Check limits 
    if player["AB"] < 170:
        return 0
    # Calculate the bonus
    bonus = 0
    bonus += player.HR * (1/6)
    bonus += player.RBI * (1/10)
    bonus += player.R * (1/10)
    bonus += player.H * (1/15)
    bonus += player.SBN * (1/2)
    bonus -= player.SO * (1/10)
    bonus += player.BB * (1/2)
    bonus += player.AVG * (100/3)
    bonus += player.OBP * (100/4)
    bonus += player.SLG * (100/5)

    bonus *= 1/10
    
    # bonus *= player.AB * 1/500 
    return bonus

## Pitchers
def get_perf_bonus_pitchers(player_id, pitch_stats):
    # Check if player is in list
    if not (player_id in pitch_stats.index):
        return 0
    # Get the player's stats
    player = pitch_stats.loc[player_id]
    # Check limits 
    # Identify SP and RP
    sp = "SP" in player["Position"]  
    if sp:
        if player["IP"] < 60:
            return 0
    else:
        if player["IP"] < 30:
            return 0
    # Calculate some extra stats
    ER = player.ERA * player.IP / 9
    K = player["K/9"] * player.IP / 9 
    # Calculate the bonus
    bonus = 0
    # bonus += player.IP * 1/18
    bonus += (200 - player.H) / 10 if sp else (80 - player.H) / 10 
    bonus += (80 - player.BB) / 10 if sp else  (50 - player.BB) / 10
    bonus += K *1/18 if sp else K * 1/8 
    bonus += (100 - ER) / 10 if sp else (40 - ER) / 10
    bonus += player.RA * 1/6.5
    bonus += player.QS * 1/18
    bonus += (player.NS * 1/20) 
    bonus += (player["NS.1"] * 1/30)
    
    bonus *= 1/8
    bonus *= player.IP * 1/180 if sp else player.IP * 1/90

    return bonus

# Assign a cost to each player
def player_cost(bats, pitchers, bats_stats, pitch_stats):
    # Create a dataframe with player names, id, salary and position
    if "Status" in bats.columns:
        players = pd.concat([bats[["Player", "Status", "ID", "Salary", "Position"]], 
                        pitchers[["Player", "Status", "ID", "Salary", "Position"]]])
    else:
        players = pd.concat([bats[["Player", "ID", "Salary", "Position"]], 
                        pitchers[["Player", "ID", "Salary", "Position"]]])
    
    # Add the base cost
    # players["cost_base"] = players["Salary"].apply(get_cost_base)
    players["cost_base"] = 30 - players["Salary"] + 1

    # Add the performance bonus
    # Batting
    players["perf_bonus_bat_2"] =  players.ID.apply(lambda x: get_perf_bonus_bats(x,bats_stats))
    # Normalize usuing Z-score
    players["perf_bonus_bat_2"] = (players["perf_bonus_bat_2"] - players["perf_bonus_bat_2"].mean()) / players["perf_bonus_bat_2"].std()
    # Set negative values to 0
    players.loc[players.perf_bonus_bat_2 < 0, "perf_bonus_bat_2"] = 0
    # Pitching
    players["perf_bonus_pitch_2"] =  players.ID.apply(lambda x: get_perf_bonus_pitchers(x, pitch_stats)) 
    # Normalize usuing Z-score
    players["perf_bonus_pitch_2"] = (players["perf_bonus_pitch_2"] - players["perf_bonus_pitch_2"].mean()) / players["perf_bonus_pitch_2"].std()
    # Set negative values to 0
    players.loc[players.perf_bonus_pitch_2 < 0, "perf_bonus_pitch_2"] = 0
    # Normalize the performance bonus for bats abd pitchers
    players["perf_bonus_bat_2"] = 10 * (players["perf_bonus_bat_2"] / players["perf_bonus_bat_2"].max())
    players["perf_bonus_pitch_2"] = 10 * (players["perf_bonus_pitch_2"] / players["perf_bonus_pitch_2"].max())
    # Add the total cost
    players["perf_bonus_2"] = players["perf_bonus_bat_2"] + players["perf_bonus_pitch_2"]
    players["cost_2"] = players["cost_base"]  + players["perf_bonus_2"]
    # Fix the cost of rookies to NaN
    players.loc[players.Status == "Min", "cost_2"] = np.nan
    return players

players_cost_2 = player_cost(bats_stats, pitch_stats, bats_stats, pitch_stats).sort_values("cost_2", ascending=False)
players_cost_2.drop_duplicates(inplace=True)

players_cost_2.sort_values("perf_bonus_2", ascending=False).head(20)

,Player,Status,ID,Salary,Position,cost_base,perf_bonus_bat_2,perf_bonus_pitch_2,perf_bonus_2,cost_2
ID,,,,,,,,,,
*02yc4*,Shohei Ohtani,TDRJ,*02yc4*,5.0,"UT,SP",26.0,6.205598,8.860425,15.066023,41.066023
*02z0s*,Aaron Judge,TDRJ,*02z0s*,4.0,OF,27.0,10.000000,0.000000,10.000000,37.000000
*000dp*,Justin Verlander,Freeman,*000dp*,1.0,SP,30.0,0.000000,10.000000,10.000000,40.000000
*041rd*,Corbin Burnes,TDRJ,*041rd*,12.0,SP,19.0,0.000000,9.684278,9.684278,28.684278
*03btv*,Sandy Alcantara,METS,*03btv*,19.0,SP,12.0,0.000000,9.261479,9.261479,21.261479
*02i3o*,Aaron Nola,Champs,*02i3o*,3.0,SP,28.0,0.000000,9.159931,9.159931,37.159931
*041qo*,Zac Gallen,Patron#5,*041qo*,8.0,SP,23.0,0.000000,9.130419,9.130419,32.130419
*04ug5*,Emmanuel Clase,Champs,*04ug5*,20.0,RP,11.0,0.000000,9.087187,9.087187,20.087187
*01spo*,Gerrit Cole,La Maqui,*01spo*,1.0,SP,30.0,0.000000,9.024299,9.024299,39.024299


In [158]:
players_cost_2[players_cost_2.Position.apply(lambda x: "P" in x)].sort_values("perf_bonus_pitch_2", ascending=False).head(30)

,Player,Status,ID,Salary,Position,cost_base,perf_bonus_bat_2,perf_bonus_pitch_2,perf_bonus_2,cost_2
ID,,,,,,,,,,
*000dp*,Justin Verlander,Freeman,*000dp*,1.0,SP,30.0,-0.524782,3.637253,3.112471,33.112471
*041rd*,Corbin Burnes,TDRJ,*041rd*,12.0,SP,19.0,-0.524782,3.522417,2.997634,21.997634
*03btv*,Sandy Alcantara,METS,*03btv*,19.0,SP,12.0,-0.524782,3.368634,2.843852,14.843852
*02i3o*,Aaron Nola,Champs,*02i3o*,3.0,SP,28.0,-0.524782,3.331698,2.806916,30.806916
*041qo*,Zac Gallen,Patron#5,*041qo*,8.0,SP,23.0,-0.524782,3.320964,2.796182,25.796182
*04ug5*,Emmanuel Clase,Champs,*04ug5*,20.0,RP,11.0,-0.524782,3.305240,2.780457,13.780457
*01spo*,Gerrit Cole,La Maqui,*01spo*,1.0,SP,30.0,-0.524782,3.282366,2.757583,32.757583
*02c2r*,Carlos Rodon,Freeman,*02c2r*,20.0,SP,11.0,-0.524782,3.239105,2.714323,13.714323
*02yc4*,Shohei Ohtani,TDRJ,*02yc4*,5.0,"UT,SP",26.0,2.935799,3.222760,6.158560,32.158560


In [157]:
players_cost_2[players_cost_2.Position.apply(lambda x: "P" not in x)].sort_values("perf_bonus_bat_2", ascending=False).head(30)

,Player,Status,ID,Salary,Position,cost_base,perf_bonus_bat_2,perf_bonus_pitch_2,perf_bonus_2,cost_2
ID,,,,,,,,,,
*02z0s*,Aaron Judge,TDRJ,*02z0s*,4.0,OF,27.0,4.730888,-0.548246,4.182643,31.182643
*03ynj*,Juan Soto,Tomy,*03ynj*,8.0,OF,23.0,4.227037,-0.548246,3.678792,26.678792
*01cgo*,Freddie Freeman,Freeman,*01cgo*,1.0,"1B,INF",30.0,3.856623,-0.548246,3.308378,33.308378
*01ual*,Paul Goldschmidt,DN,*01ual*,2.0,"1B,INF",29.0,3.627531,-0.548246,3.079286,32.079286
*01ub6*,Jose Ramirez,Champs,*01ub6*,1.0,"3B,INF",30.0,3.545042,-0.548246,2.996796,32.996796
*04355*,Yordan Alvarez,Champs,*04355*,12.0,OF,19.0,3.423879,-0.548246,2.875634,21.875634
*024m0*,Jose Altuve,La Maqui,*024m0*,1.0,"2B,INF",30.0,3.356338,-0.548246,2.808092,32.808092
*02w7p*,Alex Bregman,sfernand,*02w7p*,2.0,"3B,INF",29.0,3.345161,-0.548246,2.796915,31.796915
*041pz*,Pete Alonso,Bobby,*041pz*,12.0,"1B,INF",19.0,3.217484,-0.548246,2.669238,21.669238


# Tercera Opcion
Usar el performace bonus como base y la ronda para normalizar el valor del jugador.

In [97]:
def third_option(bats, pitchers, bats_stats, pitch_stats):
    # Create a dataframe with player names, id, salary and position
    if "Status" in bats.columns:
        players = pd.concat([bats[["Player", "Status", "ID", "Salary", "Position"]], 
                        pitchers[["Player", "Status", "ID", "Salary", "Position"]]])
    else:
        players = pd.concat([bats[["Player", "ID", "Salary", "Position"]], 
                        pitchers[["Player", "ID", "Salary", "Position"]]])
    
    # Add the performance bonus
    # Batting
    players["perf_bonus_bat_3"] =  players.ID.apply(lambda x: get_perf_bonus_bats(x,bats_stats))
    # Pitching
    players["perf_bonus_pitch_3"] =  players.ID.apply(lambda x: get_perf_bonus_pitchers(x, pitch_stats))

    # Add the performance bonus
    players["perf_bonus_3"] = players["perf_bonus_bat_3"] + players["perf_bonus_pitch_3"]

    # Normalize the performance bonus using salary
    players["cost_3"] = players["perf_bonus_3"] / players["Salary"]

    return players


third_option(bats, pitchers, bats_stats, pitch_stats).sort_values("cost_3", ascending=False)

,Player,Status,ID,Salary,Position,perf_bonus_bat_3,perf_bonus_pitch_3,perf_bonus_3,cost_3
8,Mike Trout,Act,*01s8u*,1,OF,76.291667,0.000000,76.291667,76.291667
5,Francisco Lindor,Act,*02611*,2,"SS,INF",83.821667,0.000000,83.821667,41.910833
7,Aaron Judge,Act,*02z0s*,4,OF,127.745000,0.000000,127.745000,31.936250
0,Shohei Ohtani,Act,*02yc4*,5,"UT,SP",84.113333,39.176642,123.289975,24.657995
5,Edwin Diaz,Act,*02my5*,3,RP,0.000000,39.752449,39.752449,13.250816
14,Trevor Story,IR,*02bqh*,5,"2B,SS,INF",54.721667,0.000000,54.721667,10.944333
9,Kyle Tucker,Act,*03pir*,9,OF,89.010000,0.000000,89.010000,9.890000
6,Ian Happ,Act,*03nmw*,13,OF,70.550000,0.000000,70.550000,5.426923
6,Josh Hader,Act,*02n96*,6,RP,0.000000,30.551496,30.551496,5.091916
4,Gavin Lux,Act,*041ai*,14,"2B,SS,INF,OF",61.863333,0.000000,61.863333,4.418810


In [98]:
players_cost_3 = third_option(bats_stats, pitch_stats, bats_stats, pitch_stats).sort_values("cost_3", ascending=False)
players_cost_3.drop_duplicates(inplace=True)

In [99]:
players_cost_3

,Player,Status,ID,Salary,Position,perf_bonus_bat_3,perf_bonus_pitch_3,perf_bonus_3,cost_3
ID,,,,,,,,,
*01cgo*,Freddie Freeman,Freeman,*01cgo*,1.0,"1B,INF",106.495000,0.0,106.495000,106.495000
*01ub6*,Jose Ramirez,Champs,*01ub6*,1.0,"3B,INF",98.921667,0.0,98.921667,98.921667
*024m0*,Jose Altuve,La Maqui,*024m0*,1.0,"2B,INF",94.335000,0.0,94.335000,94.335000
*01svq*,Manny Machado,DN,*01svq*,1.0,"3B,INF",88.903333,0.0,88.903333,88.903333
*02btf*,Mookie Betts,sfernand,*02btf*,1.0,OF,86.226667,0.0,86.226667,86.226667
...,...,...,...,...,...,...,...,...,...
*0001t*,Jose Valverde,FA,*0001t*,NaN,RP,0.000000,0.0,0.000000,NaN
*02bs8*,Logan Verrett,FA,*02bs8*,NaN,RP,0.000000,0.0,0.000000,NaN
*04ov2*,Austin Warner,FA,*04ov2*,NaN,SP,0.000000,0.0,0.000000,NaN


In [100]:
final = pd.merge(players_cost_2.reset_index(drop=True, inplace=False),
        players_cost_3.reset_index(drop=True, inplace=False), 
    on=["ID", "Player", "Status", "ID", "Salary", "Position"], suffixes=("_2", "_3"))

In [101]:
final = final[final.Status != "FA"]
final.dropna(inplace=True)
final[["Player", "Status","Salary", "cost_base", "cost_2"]].head(30)

,Player,Status,Salary,cost_base,cost_2
0,Shohei Ohtani,TDRJ,5.0,26.0,40.491310
1,Freddie Freeman,Freeman,1.0,30.0,38.336530
2,Justin Verlander,Freeman,1.0,30.0,38.327163
3,Jacob deGrom,DN,1.0,30.0,38.194651
4,Max Scherzer,Champs,1.0,30.0,38.002521
5,Jose Ramirez,Champs,1.0,30.0,37.743682
6,Clayton Kershaw,Patron#5,1.0,30.0,37.704646
7,Jose Altuve,La Maqui,1.0,30.0,37.384633
8,Gerrit Cole,La Maqui,1.0,30.0,37.336781
9,Aaron Judge,TDRJ,4.0,27.0,37.000000


# Budget

In [102]:
my_t = final[final["Status"] == "TDRJ"]
# Import my ranks
my_ranks = pd.read_csv("./data/my_ranks.csv")
my_ranks = my_ranks[my_ranks["Status"] == "TDRJ"]
# merge the two dataframes using Player as the key
my_t = pd.merge(my_t, my_ranks[["Player", "MEAN"]], on="Player", how="left")
my_t.dropna(inplace=True)


my_t

,Player,Status,ID,Salary,Position,cost_base,perf_bonus_bat_2,perf_bonus_pitch_2,cost_2,perf_bonus_bat_3,perf_bonus_pitch_3,perf_bonus_3,cost_3,MEAN
0,Shohei Ohtani,TDRJ,*02yc4*,5.0,"UT,SP",26.0,6.584472,7.906838,40.491310,84.113333,39.176642,123.289975,24.657995,129.576175
1,Aaron Judge,TDRJ,*02z0s*,4.0,OF,27.0,10.000000,0.000000,37.000000,127.745000,0.000000,127.745000,31.936250,83.007468
2,Edwin Diaz,TDRJ,*02my5*,3.0,RP,28.0,0.000000,8.023050,36.023050,0.000000,39.752449,39.752449,13.250816,64.260499
3,Mike Trout,TDRJ,*01s8u*,1.0,OF,30.0,5.972184,0.000000,35.972184,76.291667,0.000000,76.291667,76.291667,70.671293
4,Francisco Lindor,TDRJ,*02611*,2.0,"SS,INF",29.0,6.561640,0.000000,35.561640,83.821667,0.000000,83.821667,41.910833,56.526834
5,Josh Hader,TDRJ,*02n96*,6.0,RP,25.0,0.000000,6.166065,31.166065,0.000000,30.551496,30.551496,5.091916,60.383603
8,Kyle Tucker,TDRJ,*03pir*,9.0,OF,22.0,6.967787,0.000000,28.967787,89.010000,0.000000,89.010000,9.890000,71.553214
9,Anthony Rendon,TDRJ,*01tt9*,3.0,"3B,INF",28.0,0.000000,0.000000,28.000000,0.000000,0.000000,0.000000,0.000000,50.144828
10,Corbin Burnes,TDRJ,*041rd*,12.0,SP,19.0,0.000000,7.627845,26.627845,0.000000,37.794296,37.794296,3.149525,68.718840
11,Lucas Giolito,TDRJ,*02i37*,12.0,SP,19.0,0.000000,5.124664,24.124664,0.000000,25.391580,25.391580,2.115965,48.731489


In [103]:
# Create a function that takes a cost system and a budget and returns the best keeper list
def best_keepers(data, team_name, cost_system, budget, limit, limit_firm=True):
    """_summary_

    Args:
        cost_system (str): "2" or "3"
        budget (int): The budget to use
        limit (int): The (max) number of players to keep
        limit_firm (bool): If True, the limit is firm. If False, the limit is a maximum
    """


    # The data for the Knapsack problem
    # P is total weight capacity of sack
    # weights and utilities are also specified
    P = budget
    N = limit

    data_sub = data[data["Status"] == team_name]
    # Import my ranks
    my_ranks = pd.read_csv("./data/my_ranks.csv")
    my_ranks = my_ranks[my_ranks["Status"] == team_name]
    # merge the two dataframes using Player as the key
    data_sub = pd.merge(data_sub, my_ranks[["Player", "MEAN"]], on="Player", how="left")
    data_sub.dropna(inplace=True)


    cost = "cost_" + cost_system
    weights = np.array(data_sub[cost])
    utilities = np.array(data_sub.MEAN)

    # The variable we are solving for
    selection = cvxpy.Variable(len(weights),  boolean = True)


    # The sum of the weights should be less than or equal to P
    weight_constraint = weights @ selection <= P
    if limit_firm:
        number_constraint = sum( selection ) == N
    else:
        number_constraint = sum( selection ) <= N


    # Our total utility is the sum of the item utilities
    total_utility = utilities @ selection

    # We tell cvxpy that we want to maximize total utility 
    # subject to weight_constraint. All constraints in 
    # cvxpy must be passed as a list
    knapsack_problem = cvxpy.Problem(cvxpy.Maximize(total_utility), [weight_constraint, number_constraint])

    # Solving the problem
    knapsack_problem.solve(solver=cvxpy.GLPK_MI)
    
    return data_sub.loc[[ i == 1.0 for i in selection.value], ["Player", "Position", "Salary", cost]]

best_keepers(final, "Freeman", "2", 200, 10)

,Player,Position,Salary,cost_2
0,Freddie Freeman,"1B,INF",1.0,38.336530
1,Justin Verlander,SP,1.0,38.327163
2,Trea Turner,"2B,SS,INF",1.0,36.713505
8,Vladimir Guerrero Jr.,"1B,INF",8.0,29.482315
13,Joe Musgrove,SP,17.0,20.852537
14,Carlos Rodon,SP,20.0,18.658259
24,Camilo Doval,RP,30.0,7.428769
27,Christian Walker,"1B,INF",30.0,7.292353
35,Hunter Brown,"SP,RP",30.0,1.000000
40,Triston Casas,"1B,3B,INF",30.0,1.000000


In [104]:
best_keepers(final, "Freeman", "2", 130, 10)

,Player,Position,Salary,cost_2
0,Freddie Freeman,"1B,INF",1.0,38.336530
8,Vladimir Guerrero Jr.,"1B,INF",8.0,29.482315
14,Carlos Rodon,SP,20.0,18.658259
20,Alec Bohm,"1B,3B,INF",25.0,10.490848
24,Camilo Doval,RP,30.0,7.428769
27,Christian Walker,"1B,INF",30.0,7.292353
28,Michael King,"SP,RP",24.0,7.000000
29,Brendan Donovan,"1B,2B,3B,INF,OF",30.0,6.351547
35,Hunter Brown,"SP,RP",30.0,1.000000
40,Triston Casas,"1B,3B,INF",30.0,1.000000


In [105]:
final.Status.value_counts()

Freeman     44
Champs      44
TDRJ        43
Patron#5    43
sfernand    42
DN          41
Pinstrip    41
Tomy        41
Canha Ge    40
La Maqui    39
METS        38
Bobby       35
Name: Status, dtype: int64

In [106]:
best_keepers(final, "Canha Ge", "2", 130, 10, False)

,Player,Position,Salary,cost_2
6,Brandon Woodruff,SP,10.0,28.323824
7,Bryan Reynolds,OF,12.0,24.483189
11,Spencer Strider,"SP,RP",19.0,20.420262
15,Luis Robert,OF,20.0,14.989328
17,Jhoan Duran,"SP,RP",30.0,10.699423
19,Jason Adam,RP,30.0,8.022543
21,Joe Ryan,SP,30.0,7.693312
24,George Kirby,SP,30.0,7.585037
27,Taylor Ward,OF,30.0,6.771394
37,Corbin Carroll,OF,30.0,1.000000


In [107]:
data_sub = final[final["Status"] == "Pinstrip"]
# Import my ranks
my_ranks = pd.read_csv("./data/my_ranks.csv")
my_ranks = my_ranks[my_ranks["Status"] == "Pinstrip"]
# merge the two dataframes using Player as the key
data_sub = pd.merge(data_sub, my_ranks[["Player", "MEAN"]], on="Player", how="left")
data_sub.dropna(inplace=True)


In [108]:

cost = "cost_" + "2"
weights = np.array(data_sub[cost])
utilities = np.array(data_sub.MEAN)

# The variable we are solving for
selection = cvxpy.Variable(len(weights),  boolean = True)


# The sum of the weights should be less than or equal to P
weight_constraint = weights @ selection <= 130
if False:
    number_constraint = sum( selection ) == 10
else:
    number_constraint = sum( selection ) <= 10


# Our total utility is the sum of the item utilities
total_utility = utilities @ selection

# We tell cvxpy that we want to maximize total utility 
# subject to weight_constraint. All constraints in 
# cvxpy must be passed as a list
knapsack_problem = cvxpy.Problem(cvxpy.Maximize(total_utility), [weight_constraint, number_constraint])

# Solving the problem
knapsack_problem.solve(solver=cvxpy.GLPK_MI)

459.6162467835595

In [109]:
final[final.Position.apply(lambda x: "P" in x)].sort_values("perf_bonus_pitch_2", ascending=False).head(30)

,Player,Status,ID,Salary,Position,cost_base,perf_bonus_bat_2,perf_bonus_pitch_2,cost_2,perf_bonus_bat_3,perf_bonus_pitch_3,perf_bonus_3,cost_3
278,Felix Bautista,Tomy,*05t64*,30.0,"SP,RP",1.0,0.000000,10.000000,11.000000,0.000000,49.547800,49.547800,1.651593
281,Alexis Diaz,DN,*03qqf*,30.0,"SP,RP",1.0,0.000000,9.735803,10.735803,0.000000,48.238760,48.238760,1.607959
282,Jhoan Duran,Canha Ge,*04apj*,30.0,"SP,RP",1.0,0.000000,9.699423,10.699423,0.000000,48.058505,48.058505,1.601950
286,Penn Murfee,Champs,*05a07*,30.0,"SP,RP",1.0,0.000000,9.481522,10.481522,0.000000,46.978856,46.978856,1.565962
288,Jorge Lopez,TDRJ,*02brf*,30.0,"SP,RP",1.0,0.000000,9.399969,10.399969,0.000000,46.574779,46.574779,1.552493
289,Matt Moore,Pinstrip,*01fug*,30.0,"SP,RP",1.0,0.000000,9.386713,10.386713,0.000000,46.509098,46.509098,1.550303
147,Collin McHugh,Canha Ge,*02hy2*,17.0,"SP,RP",14.0,0.000000,9.383923,23.383923,0.000000,46.495274,46.495274,2.735016
290,Griffin Jax,Pinstrip,*0317q*,30.0,"SP,RP",1.0,0.000000,9.309604,10.309604,0.000000,46.127038,46.127038,1.537568
291,Reynaldo Lopez,Canha Ge,*03i81*,30.0,"SP,RP",1.0,0.000000,9.300267,10.300267,0.000000,46.080777,46.080777,1.536026
293,Eli Morgan,Pinstrip,*04eiw*,30.0,"SP,RP",1.0,0.000000,9.042190,10.042190,0.000000,44.802062,44.802062,1.493402
